In [10]:
import pandas as pd
data = pd.read_csv("sanfrancisco.csv")[["X","Y","Dates","Category"]]
data = data[data["Category"].isin(["LARCENY/THEFT","ASSAULT","DRUG/NARCOTIC","VANDALISM","PROSTITUTION","KIDNAPPING"])]

In [11]:
data.groupby("Category").count()

,X,Y,Dates
Category,,,
ASSAULT,76876,76876,76876
DRUG/NARCOTIC,53971,53971,53971
KIDNAPPING,2341,2341,2341
LARCENY/THEFT,174900,174900,174900
PROSTITUTION,7484,7484,7484
VANDALISM,44725,44725,44725


In [12]:
data.shape

(360297, 4)

In [13]:
174900/360297

0.4854328512310677

In [14]:
from datetime import datetime
def conversor(x):
    a = datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
    return a
def timestamp(x):
    return conversor(x).timestamp()

In [15]:
data["Dates"] = data["Dates"].apply(timestamp)

In [16]:
display(data.head())

,X,Y,Dates,Category
3,-122.426995,37.800873,1.431571e+09,LARCENY/THEFT
4,-122.438738,37.771541,1.431571e+09,LARCENY/THEFT
5,-122.403252,37.713431,1.431571e+09,LARCENY/THEFT
8,-122.508194,37.776601,1.431569e+09,LARCENY/THEFT
9,-122.419088,37.807802,1.431569e+09,LARCENY/THEFT


In [17]:
crimenes = ["LARCENY/THEFT","ASSAULT","DRUG/NARCOTIC","VANDALISM","PROSTITUTION","KIDNAPPING"]

In [18]:
def crimenes_a_num(x):
    global crimenes
    return crimenes.index(x)
def num_a_crimen(x):
    global crimenes
    return crimenes[x]

In [19]:
data["Category"] = data["Category"].apply(crimenes_a_num)

In [20]:
display(data.head())

,X,Y,Dates,Category
3,-122.426995,37.800873,1.431571e+09,0
4,-122.438738,37.771541,1.431571e+09,0
5,-122.403252,37.713431,1.431571e+09,0
8,-122.508194,37.776601,1.431569e+09,0
9,-122.419088,37.807802,1.431569e+09,0


In [21]:
import numpy as np
def to_categorical(x):
    a = []
    for i in range(6):
        if x == i:
            a.append(1)
        else:
            a.append(0)
    return np.asarray(a).astype(np.int)

In [22]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
predictores = data[["Dates","X","Y"]]
a_predecir = to_categorical(data["Category"])
print(a_predecir)
X_train, X_test, y_train, y_test = train_test_split(predictores, a_predecir, test_size=0.2)

2021-09-15 19:50:51.684932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-15 19:50:51.684958: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [23]:
from sklearn import preprocessing as prepro
X_train = prepro.StandardScaler().fit_transform(X_train)
X_test = prepro.StandardScaler().fit_transform(X_test)

In [38]:
from tensorflow import keras
from tensorflow.keras import layers
NN = keras.Sequential(
    [layers.Dense(64,activation="relu",input_shape=(3,)),
    layers.Dense(32,activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(16,activation="relu"),
    layers.Dense(6,activation="softmax")]
)


In [39]:
from tensorflow.keras.utils import plot_model
plot_model(NN)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [40]:
NN.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 64)                256       
_________________________________________________________________
dense_17 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_19 (Dense)             (None, 6)                 102       
Total params: 2,966
Trainable params: 2,966
Non-trainable params: 0
_________________________________________________________________


In [41]:
NN.compile("adam", "categorical_crossentropy", metrics=["accuracy"])


In [42]:
import tensorflow as tf
NN.fit(X_train,y_train, epochs=5,batch_size=64,validation_data=(X_test,y_test))


Epoch 1/5
4504/4504 [==============================] - 10s 2ms/step - loss: 1.2694 - accuracy: 0.5002 - val_loss: 1.2249 - val_accuracy: 0.5185
Epoch 2/5
4504/4504 [==============================] - 10s 2ms/step - loss: 1.2087 - accuracy: 0.5261 - val_loss: 1.1941 - val_accuracy: 0.5401
Epoch 3/5
4504/4504 [==============================] - 10s 2ms/step - loss: 1.1877 - accuracy: 0.5394 - val_loss: 1.1734 - val_accuracy: 0.5487
Epoch 4/5
4504/4504 [==============================] - 11s 2ms/step - loss: 1.1757 - accuracy: 0.5444 - val_loss: 1.1666 - val_accuracy: 0.5481
Epoch 5/5
4504/4504 [==============================] - 12s 3ms/step - loss: 1.1688 - accuracy: 0.5485 - val_loss: 1.1598 - val_accuracy: 0.5532


In [33]:
#NN.save("red.h5")